In [11]:
import sys
sys.path.append("../code/")
import pandas as pd
import numpy as np
import h5py
import util_hdf5, util_Stats
# import tensorflow as tf

# gpus = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_visible_devices(gpus[2], 'GPU')

In [12]:
data_dic = {
#     'British-insample': [
#         '/vol/bmd/yanyul/GitHub/ptrs-ukb/output/data_split/British-training-1.txt',
#         '/vol/bmd/yanyul/UKB/predicted_expression_tf2/ukb_imp_x_ctimp_Whole_Blood_British.hdf5'
#     ],
    'British-test-1': [
        '/vol/bmd/yanyul/GitHub/ptrs-ukb/output/data_split/British-test-1.txt',
        '/vol/bmd/yanyul/UKB/predicted_expression_tf2/ukb_imp_x_ctimp_Whole_Blood_British.hdf5'
    ],
    'British-validation-1': [
        '/vol/bmd/yanyul/GitHub/ptrs-ukb/output/data_split/British-validation-1.txt',
        '/vol/bmd/yanyul/UKB/predicted_expression_tf2/ukb_imp_x_ctimp_Whole_Blood_British.hdf5'
    ],
    'Chinese': [
        '/vol/bmd/yanyul/GitHub/ptrs-ukb/output/data_split/Chinese.txt',
        '/vol/bmd/yanyul/UKB/predicted_expression_tf2/ukb_imp_x_ctimp_Whole_Blood_Chinese.hdf5'
    ],
    'Indian': [
        '/vol/bmd/yanyul/GitHub/ptrs-ukb/output/data_split/Indian.txt',
        '/vol/bmd/yanyul/UKB/predicted_expression_tf2/ukb_imp_x_ctimp_Whole_Blood_Indian.hdf5'
    ],
    'African': [
        '/vol/bmd/yanyul/GitHub/ptrs-ukb/output/data_split/African.txt',
        '/vol/bmd/yanyul/UKB/predicted_expression_tf2/ukb_imp_x_ctimp_Whole_Blood_African.hdf5'
    ]
}
scheme_yaml = '../misc_files/data_scheme.yaml'
ptrs_types = ['naive', 'ldblock', 'mashr']

In [13]:
feature_dic = util_hdf5.read_yaml(scheme_yaml)
with h5py.File(data_dic['British-test-1'][1], 'r') as f:
    features = f['columns_y'][:].astype('str')
    sample_size = f['y'].shape[0]
    y = f['y'][:]
covar_indice = np.where(np.isin(features, feature_dic['covar_names']))[0]
trait_indice = np.where(np.isin(features, feature_dic['outcome_names']))[0]


In [14]:
filename = f'/vol/bmd/yanyul/UKB/ptrs-tf/prs-from-cri/prs_gcp_subset1.tsv.bgz'
df_prs = pd.read_csv(filename, sep = '\t', compression = 'gzip')

In [15]:
df_prs['s'] = df_prs['s'].astype('str')

In [16]:
df_prs

,s,pval_thres_subset1_x_basophil_x_5e-08,pval_thres_subset1_x_basophil_x_1e-07,pval_thres_subset1_x_basophil_x_1e-06,pval_thres_subset1_x_basophil_x_1e-05,pval_thres_subset1_x_basophil_x_0.0001,pval_thres_subset1_x_basophil_x_0.001,pval_thres_subset1_x_basophil_x_0.01,pval_thres_subset1_x_basophil_x_0.1,pval_thres_subset1_x_basophil_x_0.05,...,pval_thres_subset1_x_wbc_x_1e-07,pval_thres_subset1_x_wbc_x_1e-06,pval_thres_subset1_x_wbc_x_1e-05,pval_thres_subset1_x_wbc_x_0.0001,pval_thres_subset1_x_wbc_x_0.001,pval_thres_subset1_x_wbc_x_0.01,pval_thres_subset1_x_wbc_x_0.1,pval_thres_subset1_x_wbc_x_0.05,pval_thres_subset1_x_wbc_x_0.5,pval_thres_subset1_x_wbc_x_1.0
0,5595764,0.002818,0.000548,-0.002355,-0.039979,-0.078601,-0.148310,-0.299340,-0.53566,-0.541310,...,-2.92460,-3.25670,-3.97300,-3.285300,-5.02040,-12.5550,-4.2222,-12.52500,3.86150,2.44830
1,5172041,0.015853,0.016976,0.017596,-0.013238,-0.004562,-0.054524,-0.107530,-0.47255,-0.513920,...,3.76400,3.82460,3.73170,5.267400,1.37810,-8.5345,4.7830,-6.47590,-0.17849,-1.05330
2,2266650,-0.002270,-0.004951,-0.003398,-0.041235,-0.078107,-0.233080,-0.557460,-0.99163,-1.000700,...,1.51480,0.27255,0.42501,-1.086700,-7.82310,-20.0810,-8.6108,-17.09600,-2.61090,-0.17238
3,4860485,0.016499,0.012364,0.005550,-0.003007,-0.040464,-0.148360,-0.214210,-0.36219,-0.494810,...,1.83650,1.03510,1.12770,0.541390,-1.18630,-8.8609,6.6322,0.80335,3.54890,4.39960
4,3131191,0.007486,0.005260,0.005107,-0.022466,-0.042266,-0.111090,-0.162860,-0.56544,-0.596290,...,0.96699,0.72013,0.90809,0.195640,-3.30870,-15.7870,-10.5090,-15.81600,-20.91700,-22.59300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18945,3686944,0.064743,0.060037,0.069216,0.062796,0.087676,0.113910,0.093876,0.23213,0.036713,...,9.26540,8.74330,9.64170,11.168000,12.26600,11.1650,35.0150,25.84000,51.01700,53.03300
18946,3511234,-0.003548,-0.006058,-0.014378,-0.046885,-0.098082,-0.189350,-0.366180,-0.86207,-0.759500,...,-1.57250,-2.39530,-0.84165,0.038073,-2.92490,-7.2195,15.7670,3.61950,22.51700,23.49500
18947,2969425,-0.020088,-0.026094,-0.026408,-0.058261,-0.068467,-0.124990,-0.210930,-0.55080,-0.548030,...,3.60080,2.92710,3.79680,3.784100,0.33745,-9.6559,3.9382,-1.11710,-1.93130,-2.38730
18948,2874543,-0.003687,-0.006192,-0.005742,-0.042782,-0.063477,-0.152820,-0.315810,-0.54616,-0.610200,...,0.39588,0.30329,-0.89967,0.500000,-2.23630,-12.8540,-4.5327,-9.78390,-7.19720,-9.59400


In [17]:
# load data 
import re
collect_dic = {}
for pop in data_dic.keys():
    collect_dic[pop] = {}
    print(f'{pop}')
    indiv_list = data_dic[pop][0]
    pred_expr = data_dic[pop][1]
    reference_mat = pd.read_csv(indiv_list, sep = ' ')
#     if pop == 'British-insample':
#         reference_mat = reference_mat.head(5000)
    reference_mat = reference_mat.rename(columns = {'FID': 'indiv'})
    reference_mat = reference_mat.drop(['IID'], axis = 1)
    reference_mat['indiv'] = reference_mat['indiv'].astype('str')
    with h5py.File(pred_expr, 'r') as f:
        features = f['columns_y'][...].astype('str')
        indiv_ids = f['rows'][...].astype('str')
    data_scheme, _ = util_hdf5.build_data_scheme(
        pred_expr, 
        scheme_yaml, 
        batch_size = 20000, 
        inv_norm_y = True
    )
    ymat = np.empty((0, data_scheme.get_num_covariate() + data_scheme.get_num_outcome()))
    for _, y in data_scheme.dataset:
        ymat = np.concatenate((ymat, y), axis = 0)
    g = pd.DataFrame(ymat)
    g['indiv'] = indiv_ids
    data_mat = reference_mat.join(g.set_index('indiv'), on = 'indiv').iloc[:, reference_mat.shape[1]:]
    ymat = data_mat.to_numpy()

    ## here
    covar = ymat[:, data_scheme.covariate_indice]
    y = ymat[:, data_scheme.outcome_indice]    

    prs_collector = np.empty((reference_mat.shape[0], 17, 11))
    for i in range(len(features[data_scheme.outcome_indice])):
        trait = features[data_scheme.outcome_indice][i]
        cols = []
        for j in list(df_prs.columns):
            if f'pval_thres_subset1_x_{trait}_x_' in j:
                cols.append(j)
        prs_mat = df_prs[['s'] + cols]
        out_mat = reference_mat.join(prs_mat.set_index('s'), on = 'indiv').iloc[:, 1:]
        out_mat = out_mat.to_numpy()
        prs_collector[:, i, :] = out_mat
    prs_names = [ re.sub(f'pval_thres_subset1_x_{trait}_x_', '', i) for i in cols ]
    collect_dic[pop] = [util_Stats.quick_partial_r2(covar, y, prs_collector), prs_names]

British-test-1
(26, 26)
British-validation-1
(26, 26)
Chinese
(26, 26)
Indian
(26, 26)
African
(26, 26)


In [18]:
def _pr2_format(ele, features, name, alpha, lambda_):
    nlambda = ele.shape[1]
    ntrait = ele.shape[0]
    ele_seq = np.reshape(ele, (nlambda * ntrait), order = 'C')
    lambda_seq = np.tile(lambda_, ntrait)
    f_seq = np.repeat(features, nlambda)
    return pd.DataFrame({'partial_r2': ele_seq, 'trait': f_seq, 'sample': name, 'ptrs_type': alpha, 'ptrs_cutoff': lambda_seq})
df = pd.DataFrame({'partial_r2': [], 'trait': [], 'sample': [], 'ptrs_type': [], 'ptrs_cutoff': []})
for pop in collect_dic.keys():
    df = pd.concat((df, _pr2_format(collect_dic[pop][0], features[trait_indice], pop, 'prs', collect_dic[pop][1])))

In [19]:
df

,partial_r2,trait,sample,ptrs_type,ptrs_cutoff
0,0.159664,height,British-test-1,prs,5e-08
1,0.161398,height,British-test-1,prs,1e-07
2,0.171793,height,British-test-1,prs,1e-06
3,0.177768,height,British-test-1,prs,1e-05
4,0.185643,height,British-test-1,prs,0.0001
...,...,...,...,...,...
182,0.000018,basophil,African,prs,0.01
183,0.000271,basophil,African,prs,0.1
184,0.000019,basophil,African,prs,0.05
185,0.000221,basophil,African,prs,0.5


In [20]:
df.to_csv('/vol/bmd/yanyul/UKB/ptrs-tf/models/partial_r2-prs.subset1_British.tsv', sep = '\t', index = False)